In [2]:
from llm_client import invoke_llm
from features.iternary_generation.llm_parser import llm_parse_user_input

In [3]:
prompt = "Plan a 5-day foodie + nightlife trip to Goa with 40k budget"

In [4]:
from features.iternary_generation.llm_parser import llm_parse_user_input
from features.iternary_generation.iternary_generator import generate_itinerary
from features.iternary_generation.basic_tag_personalization import apply_personalization
from features.maps_scrapper.scraper import fetch_google_reviews
from features.maps_scrapper.scraper import preprocess_google_reviews
from features.reddit_scraper.summarizer import summarize_places

DEBUG: Found .env at: d:\tripPlanner\trip-planner-genai\backend\.env


In [5]:
prompt = "Plan a 5-day foodie + nightlife trip to Goa with 40k budget"

In [6]:
parsed = llm_parse_user_input(prompt)
print("Parsed Input:", parsed)

place = parsed["location"]

Prompt: 

        Extract the following fields from the trip request:
        - Location (city or region in India)
        - Duration in days
        - Budget in INR
        - Themes (heritage, nightlife, adventure, food, nature, general)

        Input: "Plan a 5-day foodie + nightlife trip to Goa with 40k budget"
        
        Return ONLY valid JSON (no markdown, no explanation).
        {
        "location": "...",
        "duration_days": <int>,
        "budget": <int>,
        "themes": [ ... ]
        }
    
Client Created successfully
```json
{
"location": "Goa",
"duration_days": 5,
"budget": 40000,
"themes": ["food", "nightlife"]
}
```
Parsed Input: {'location': 'Goa', 'duration_days': 5, 'budget': 40000, 'themes': ['food', 'nightlife']}


In [7]:
import sys
print(sys.executable)
!{sys.executable} -m pip install spacy
!{sys.executable} -m spacy download en_core_web_sm


d:\tripPlanner\trip-planner-genai\backend\.venv\Scripts\python.exe



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     --------------------------------------- 0.1/12.8 MB 154.0 kB/s eta 0:01:23
     --------------------------------------- 0.1/12.8 MB 154.0 kB/s eta 0:01:23
     --------------------------------------- 0.1/12.8 MB 136.5 kB/s eta 0:01:34
     --------------------------------------- 0.1/12.8 MB 163.8 kB/s eta 0:01:18
     --------------------------------------- 0.1/12.8 MB 163.8 kB/s eta 0:01:18
     -------------------


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
import spacy
nlp = spacy.load("en_core_web_sm")


In [9]:
def extract_spots_from_itinerary(itinerary):
    spots = set()
    for day in itinerary:
        for key in ["morning", "afternoon", "evening"]:
            if key in day:
                doc = nlp(day[key])
                for ent in doc.ents:
                    if ent.label_ in ["GPE", "LOC", "FAC"]:  # city, location, facility
                        spots.add(ent.text)
    return list(spots)

In [10]:
initial_summary = "Popular attractions and tips for visitors."
initial_itinerary = generate_itinerary(parsed, initial_summary)

Client Created successfully
```json
{
  "itinerary": [
    {
      "day": 1,
      "morning": "Arrive at Goa Airport (GOI). Take a pre-booked taxi to your accommodation in North Goa (e.g., Anjuna/Vagator). Check-in and relax. Grab a light Goan breakfast at a local 'udupi' style eatery for cheap and authentic idli/dosa.",
      "afternoon": "Head to Anjuna Beach. Explore the flea market (if it's a Wednesday, otherwise browse local stalls). Have a leisurely Goan seafood thali (fish curry rice) at a shack right on the beach. Reddit users often recommend trying the 'fish thali' for an authentic Goan lunch.",
      "evening": "Experience the vibrant nightlife of Anjuna. Start with sundowners at Curlies Beach Shack, known for its lively atmosphere. Later, head to Shiva Valley for a more trance/rave experience, or enjoy live music at one of the many pubs. Dinner will be at the beach shacks, focusing on fresh seafood BBQ.",
      "estimated_cost": 5500
    },
    {
      "day": 2,
      "morni

In [11]:
spots = extract_spots_from_itinerary(initial_itinerary["itinerary"])

In [12]:
print("Extracted Spots:", spots)

Extracted Spots: ['the Mandovi River', 'North Goa', 'Calangute', 'Ponda', 'Mambo', 'Anjuna Beach', 'Vagator Beach', 'Curlies Beach Shack', 'Anjuna', 'Goa', 'Goa Airport', 'Black Sheep Bistro', 'Candolim Beach']


In [13]:
# 4️⃣ Fetch Google reviews for each spot
all_reviews_text = []
for spot in spots:
    reviews = fetch_google_reviews(spot, limit=5)
    all_reviews_text.append(reviews)

DEBUG: Searching for place: the Mandovi River
DEBUG SERPAPI_KEY exists: True
DEBUG: Full response keys: dict_keys(['search_metadata', 'search_parameters', 'search_information', 'local_results'])
DEBUG: Full response: {
  "search_metadata": {
    "id": "68cc27531de06496312ff729",
    "status": "Success",
    "json_endpoint": "https://serpapi.com/searches/0b49a01a91241765/68cc27531de06496312ff729.json",
    "created_at": "2025-09-18 15:37:55 UTC",
    "processed_at": "2025-09-18 15:37:55 UTC",
    "google_maps_url": "https://www.google.com/maps/search/the+Mandovi+River?hl=en",
    "raw_html_file": "https://serpapi.com/searches/0b49a01a91241765/68cc27531de06496312ff729.html",
    "total_time_taken": 0.8
  },
  "search_parameters": {
    "engine": "google_maps",
    "type": "search",
    "q": "the Mandovi River",
    "google_domain": "google.com",
    "hl": "en"
  },
  "search_information": {
    "local_results_state": "Results for exact spelling",
    "query_displayed": "the Mandovi River

In [14]:
# Flatten the nested list of reviews
flat_reviews = [review for sublist in all_reviews_text for review in sublist]

for i, r in enumerate(flat_reviews, 1):
    text = r.get("snippet") or r.get("text") or "No text"
    link = r.get("link") or "No link"
    
    print(f"Review {i}:")
    print(f"Google Link: {link}")
    print(f"Text: {text}\n")
    print("-" * 80)  # separator between reviews


Review 1:
Google Link: https://www.google.com/maps/reviews/data=!4m8!14m7!1m6!2m5!1sChdDSUhNMG9nS0VJQ0FnTURJdUpQR19BRRAB!2m1!1s0x0:0x7d208504f803b76a!3m1!1s2@1:CIHM0ogKEICAgMDIuJPG_AE%7C%7C?hl=en-US
Text: **Mandovi River, Goa**

The **Mandovi River**, often referred to as the *"Lifeline of Goa,"* is one of the most picturesque and vibrant waterways in the state. Flowing through the heart of North Goa, it offers a perfect blend of natural beauty, cultural experiences, and exciting activities.

### **Scenic Beauty & Sunset Views**
The river is surrounded by lush greenery, dotted with palm trees and traditional Goan villages. The golden hues of the sunset reflecting on the water make it a mesmerizing sight. Popular spots like **Panjim’s Riverside Promenade (Dona Paula & Miramar)** and **Reis Magos Fort** provide stunning viewpoints.

### **River Cruises & Water Activities**
One of the highlights of the Mandovi River is the **sunset cruise**. These floating casinos and dinner cruises (like

In [18]:
from features.maps_scrapper.image_scraper import fetch_place_image

In [19]:
all_images = []

for spot in spots:
    image_url = fetch_place_image(spot)  # function from SerpAPI / Unsplash / DALL·E
    all_images.append({
        "place": spot,
        "image_url": image_url
    })

# Example output
print(all_images)


[{'place': 'the Mandovi River', 'image_url': 'https://upload.wikimedia.org/wikipedia/commons/1/10/Panaji%2C_Goa%2C_India%2C_Sunset%2C_Mandovi_River.jpg'}, {'place': 'North Goa', 'image_url': 'https://upload.wikimedia.org/wikipedia/commons/2/25/North_Goa_Political_Map.png'}, {'place': 'Calangute', 'image_url': 'https://content.r9cdn.net/rimg/dimg/0b/b2/7d5f811e-city-55845-166ea36604c.jpg?width=1366&height=768&xhint=3578&yhint=1750&crop=true'}, {'place': 'Ponda', 'image_url': 'https://static.toiimg.com/photo/52416442.cms'}, {'place': 'Mambo', 'image_url': 'https://upload.wikimedia.org/wikipedia/commons/9/96/CCMDonation28.JPG'}, {'place': 'Anjuna Beach', 'image_url': 'https://upload.wikimedia.org/wikipedia/commons/c/cf/Anjuna_Beach%2C_Goa%2C_India%2C_Legendary_Curlies_beach_shack.jpg'}, {'place': 'Vagator Beach', 'image_url': 'https://dynamic-media-cdn.tripadvisor.com/media/photo-o/16/e5/03/bd/img-20181104-094252-largejpg.jpg?w=900&h=500&s=1'}, {'place': 'Curlies Beach Shack', 'image_url'

In [21]:
# Extract review text for summarization
review_texts = [r.get("snippet") or r.get("text") for r in flat_reviews if r.get("snippet") or r.get("text")]


In [22]:
# Combine all review texts into one blob (for LLM input)
text_blob = "\n".join(review_texts)

print("Combined Review Text Preview:\n", text_blob[:500], "...")  # preview first 500 chars



Combined Review Text Preview:
 **Mandovi River, Goa**

The **Mandovi River**, often referred to as the *"Lifeline of Goa,"* is one of the most picturesque and vibrant waterways in the state. Flowing through the heart of North Goa, it offers a perfect blend of natural beauty, cultural experiences, and exciting activities.

### **Scenic Beauty & Sunset Views**
The river is surrounded by lush greenery, dotted with palm trees and traditional Goan villages. The golden hues of the sunset reflecting on the water make it a mesmerizin ...


In [23]:
summary = summarize_places(text_blob, place)

Client Created successfully
**Goa Travel Recommendations:**

*   **Mandovi River:**
    *   **Activities:** Sunset cruises (some with live music, Goan dances, seafood), floating casinos (e.g., Casino Pride, Peacock Paradise), kayaking, paddleboarding, dolphin-spotting tours.
    *   **Attractions:** Panjim’s Riverside Promenade (Dona Paula & Miramar), Reis Magos Fort, Old Goa’s Churches (e.g., Basilica of Bom Jesus), Adil Shah’s Palace (Secretariat), Salim Ali Bird Sanctuary (Chorao Island) for birdwatching and crocodile sightings.
    *   **Best Time to Visit:** November to March for pleasant weather.
    *   **Tip:** Don’t miss the evening cruise with traditional Goan music and dance.

*   **Anjuna Beach:**
    *   **Activities:** Street shopping at lively shops and flea markets for handmade jewelry, ornaments, clothes, bags, and souvenirs. Enjoying beachside cafes and shacks with food and live music. Observing waves crashing against rocky edges.
    *   **Food:** Beach shacks offer 

In [24]:
itinerary = generate_itinerary(parsed, summary)
print("Generated Itinerary:", itinerary)


Client Created successfully
```json
{
  "itinerary": [
    {
      "day": 1,
      "morning": "Arrive at Goa Dabolim International Airport (GOI). Take a prepaid taxi to your accommodation in Anjuna. Settle in and have breakfast at a local cafe.",
      "afternoon": "Head to Anjuna Beach. Explore the lively shops and flea markets, browsing for handmade jewelry, clothes, and souvenirs. Enjoy a casual lunch at a beach shack.",
      "evening": "Experience the vibrant energy of Curlies at Anjuna Beach. Enjoy a sundowner with good food, drinks, and techno music, soaking in the magical sunset views.",
      "estimated_cost": 7500
    },
    {
      "day": 2,
      "morning": "Start your day with a relaxed breakfast at Artjuna Cafe. Try their highly recommended mushroom, egg, and avocado toast or eggplant shakshuka. Enjoy the relaxed vibe and art pieces.",
      "afternoon": "Explore the historical sites of Old Goa. Visit the Basilica of Bom Jesus, a UNESCO World Heritage site, and other icon